In [ ]:
import pandas as pd
import json

In [ ]:
def nest_for_json(df, groupbycol, nestcol, newcolname):
    
    new_df = (df.groupby(groupbycol)
              .apply(lambda x: x[nestcol].values)
              .drop_duplicates()
              .reset_index()
              .rename(columns={0:newcolname}))
    
    new_df[newcolname] = new_df[newcolname].apply(lambda x: list(set(x)))
    
    return new_df
    

def returning_top_tags(df, n=10):
    return(df
 .groupby(['content_audience', 'content_tag'],sort=True)
 .count()
 .reset_index()
 .groupby(['content_audience'])
 .apply(lambda x: x.nlargest(n, 'guid'))
 [['content_tag', 'guid']]
 .reset_index()
 .drop('level_1',1)
 .rename({'guid':'count'})
)

In [ ]:
df = pd.read_csv("group_content_tags.csv")


#top_tags = df.content_tag.value_counts()[1:11].index

x = returning_top_tags(df, 1)

df = df[df.content_tag.isin(x.content_tag)]
df.drop_duplicates(inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
nest_for_json(df,['guid', 'name', 'description', 'content_guid'],
             'group_tag',
             'group_tags')

In [ ]:
content = df[['content_guid' ,'subtype', 'time_created', 'content_tag', 'content_audience']]
groups = df[['guid', 'name', 'description', 'group_tag', 'content_guid']]

In [ ]:
groups_tags = (groups
 .groupby(['guid', 'name', 'description', 'content_guid'], as_index=False)
 .apply(lambda x:x['group_tag'].values)
 .drop_duplicates()
 .reset_index()
 .rename(columns={0:"group_tags"}))
groups_tags['group_tags'] = groups_tags['group_tags'].apply(lambda x: list(set(x)))

In [ ]:
groups_content_tags = (groups
 .groupby('guid', as_index=True)
 .apply(lambda x:x['content_guid'].values)
 .drop_duplicates()
 .reset_index()
 .rename(columns={0:"content_tags", 'index':'guid'}))
groups_content_tags['content_tags'] = groups_content_tags['content_tags'].apply(lambda x: list(set(x)))

In [ ]:
content_tags = (content
 .groupby(['content_guid', 'subtype', 'time_created'], as_index=False)
 .apply(lambda x:x['content_tag'].values)
 .drop_duplicates()
 .reset_index()
 .rename(columns={0:"tags"}))
content_tags['tags'] = content_tags['tags'].apply(lambda x: list(set(x)))

In [ ]:
content_audiences = (content
 .groupby(['content_guid', 'subtype', 'time_created'], as_index=False)
 .apply(lambda x: x['content_audience'].values)
 .drop_duplicates()
 .reset_index()
 .rename(columns={0:"audience"}))

content_audiences['audience'] = content_audiences['audience'].apply(lambda x: list(set(x)))

In [ ]:
nested_content = pd.merge(content_tags,content_audiences, how='left')
nested_content_dict =nested_content.drop_duplicates('content_guid').set_index('content_guid').to_dict('index')

In [ ]:
nested_content_dict

for key, value in nested_content_dict.items():
    nested_content_dict[key]['guid'] = key

In [ ]:
groups_tags = pd.merge(groups_tags, groups_content_tags)
groups_tags = groups_tags[groups_tags['content_guid'].isin(nested_content.content_guid)]

groups_dict = groups_tags.set_index('guid').to_dict('index')

In [ ]:
for key, val in groups_dict.items():
    val['children'] = []
for key, val in groups_dict.items():
    try:
        for guid in val['content_tags']:
            val['children'].append(nested_content_dict[guid])
    except:
        pass

In [ ]:
groups_json = json.dumps(groups_dict, indent=4)

In [ ]:
# Reverse method time


comm_content_tags = nest_for_json(df, 'content_audience', 'content_tag', 'content_tags')
content_groups = nest_for_json(df,['content_audience', 'content_tag'],'guid','groups')



In [ ]:
comm_content_tags.rename(columns={'content_audience':'name'}, inplace=True)

In [ ]:
comm_content_tags['free'] = True
comm_content_tags['project'] =  False
content_groups['free'] = True
content_groups['project'] = False


In [ ]:
communities_dict= comm_content_tags.to_dict('r')

In [ ]:
content_list = content_groups.set_index('content_audience').to_dict('r')

In [ ]:
#Nesting tags
for d in communities_dict:
    d['children'] = []


for item in content_list:
    content_tag = item['content_tag']
    for d in communities_dict:
        if content_tag in d['content_tags']:
            d['children'].append(item)


In [ ]:
groups_for_json = groups[['guid', 'name', 'description']].drop_duplicates()
groups_for_json['project'] = False

groups_for_json.loc[groups_for_json.sample(10).index, 'project'] = True

groups_for_json=groups_for_json[groups_for_json.project == True]

groups_dict = groups_for_json.to_dict('r')

for d in groups_dict:
    d['free'] = True

In [ ]:
#Nesting Groups
for value_ in communities_dict:
    for content in value_['children']:
        content['children'] = []


for item in groups_dict:
    for value_ in communities_dict:
        for content in value_['children']:
            if item['guid'] in content['groups']:
                
                if item not in content['children']:
                    content['children'].append(item.copy())
        
for value_ in communities_dict:
    for content in value_['children']:
        content.pop('groups', None)

In [ ]:
groups_for_json.head()

In [ ]:
final_dict = {}
final_dict['name'] = 'Communities'
final_dict['free'] = True
final_dict['project'] = False
final_dict['children'] = communities_dict

In [ ]:
with open('new_data.json', 'w') as f:
     strang = json.loads(json.dumps(final_dict, indent=4, separators=(',', ':')).replace('"content_tag"','"name"'))
     json.dump(strang, f, indent=4, separators=(',', ':'))

In [ ]:
with open('data.json', 'w') as f:
     json.dump(communities_dict, f, indent=4, separators=(',', ':'))